In [4]:
import DataFrames 
import XLSX
include("structs.jl")

LoadError: LoadError: UndefVarError: VLim not defined
in expression starting at /Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/src/Bilevel/structs.jl:79

In [5]:
# -- FETCHING THE PATH OF THE ROOT DIRECTORY --

root_dir = normpath(joinpath(@__FILE__,"..","..",".."))

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/"

In [6]:
# -- LOADING THE EXCEL FILE CONTAINING THE NETWORK TOPOLOGY --

network_data_dir = joinpath(root_dir, "NetworkModels")
XLSX_NETWORK_PATH = joinpath(network_data_dir, "network_Nahman_Peric_2S23H.xlsx")

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/NetworkModels/network_Nahman_Peric_2S23H.xlsx"

In [7]:
# -- LOADING THE EXCEL FILES CONTAINING THE DATA FOR LOAD AND PV PROFILES --

profiles_data_dir = joinpath(root_dir, "ManchesterData", "LCT_profiles")
XLSX_SUMMER_LOAD_PATH = joinpath(profiles_data_dir, "Summer_Load_Profiles.xlsx")
XLSX_WINTER_LOAD_PATH = joinpath(profiles_data_dir, "Winter_Load_Profiles.xlsx")
XLSX_PV_PATH = joinpath(profiles_data_dir, "Summer_PV_Profiles.xlsx")
XLSX_EV_PATH = joinpath(profiles_data_dir, "Winter_EV_Profiles.xlsx")
XLSX_HP_PATH = joinpath(profiles_data_dir, "Winter_EHP_Profiles.xlsx")

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/ManchesterData/LCT_profiles/Winter_EHP_Profiles.xlsx"

In [8]:
# -- DEFINITION OF THE PER UNIT BASIS --

function define_PU_basis(;BASE_POWER::Float64 = 1.0, BASE_VOLTAGE::Float64 = 34.5)
    
    BASE_CURRENT = BASE_POWER / BASE_VOLTAGE      # [kA]
    BASE_IMPEDANCE = BASE_VOLTAGE / BASE_CURRENT  # [Ohm]

    PU_basis = (base_power=BASE_POWER, 
                base_voltage=BASE_VOLTAGE,
                base_current=BASE_CURRENT, 
                base_impedance=BASE_IMPEDANCE
                )
                
    return PU_basis 
end


= 


(base_power = 1.0, base_voltage = 34.5, base_current = 0.028985507246376812, base_impedance = 1190.25)

In [9]:
# -- DEFINITION OF THE VOLTAGE LIMITS --

voltage_limits = (V_min=0.95, V_max=1.05)

(V_min = 0.95, V_max = 1.05)

In [10]:
# -- Definition of the buses -- 

""" get_network_data 

    Arguments:
    ----------
        - XLSX_NETWORK_PATH : path of xlsx file contrainint the network data
        - voltage_limits : limits on the bus voltage
        - PU_basis : per-unit basis used for the simulation
    
    Return value:
    -------------
        - A structure of type DistributionNetwork that contains the network data
"""

function get_network_data(XLSX_NETWORK_PATH::String; voltage_limits::VLIM=(V_min=0.95, V_max=1.05), PU_basis::PU_BASIS=define_PU_basis())
    
    lines = get_lines_data(XLSX_NETWORK_PATH)
    buses = get_buses_data(XLSX_NETWORK_PATH, voltage_limits)
    conductors = get_conductors_data(XLSX_NETWORK_PATH, PU_basis)

    return DistributionNetwork(lines, buses, conductors, pu_basis)
end



function get_lines_data(XLSX_NETWORK_PATH::String)
    df_line = DataFrames.DataFrame(XLSX.readtable(XLSX_NETWORK_PATH, "line"))
    L = DataFrames.nrow(df_line)
    
    edges = [Edge(l, (from_bus=df_line.from_bus[l], to_bus=df_line.to_bus[l])) for l in 1:L]
    lines = [Line(edges[l], convert(Float64, df_line.length_km)) for l in 1:L]
    
    return lines
end

function get_buses_data(XLSX_NETWORK_PATH::String)
    df_bus = DataFrames.DataFrame(XLSX.readtable(XLSX_NETWORK_PATH, "bus"))

    N = DataFrames.nrow(df_bus)            # Total number of buses
    Ns = sum(df_bus.type .== "substation") # Number of substation buses
    Nu = N - Ns                            # Number of user buses

    nodes = [Node(i, (x=convert(Float64, df_bus.x[i]), y=convert(Float64, df_bus.y[i]))) for i in 1:N]
    
    buses = [Substation(nodes[i], V_limits, convert(Float64, df_bus.S_rating_max[i])) for i in 1:Ns]
    buses = [buses; User(nodes[i], V_limits, load_profile[i]) for i in Ns+1:N]

    return buses
end


function add_load_profile(XLSX_NETWORK_PATH::String, user_bus::User, PU_basis::PU_BASIS)
    
    lines = get_lines_data(XLSX_NETWORK_PATH)
    buses = get_buses_data(XLSX_NETWORK_PATH, voltage_limits)
    conductors = get_conductors_data(XLSX_NETWORK_PATH, PU_basis)

    return DistributionNetwork(lines, buses, conductors, pu_basis)
end



UndefVarError: UndefVarError: PU_BASIS not defined